In [1]:
import cv2
import numpy as np
import os

train_path = 'dataset/training_set'
training_names = os.listdir(train_path)

image_paths = []
image_classes = []
class_id = 0


In [2]:
def imglist(path):    
    return [os.path.join(path, f) for f in os.listdir(path)]

In [3]:
from tqdm import tqdm

In [4]:

for training_name in tqdm(training_names):
    dir = os.path.join(train_path, training_name)
    class_path = imglist(dir)
    image_paths+=class_path
    image_classes+=[class_id]*len(class_path)
    class_id+=1

# Create feature extraction and keypoint detector objects
    #SIFT is not available anymore in openCV    
# Create List where all the descriptors will be stored
des_list = []


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 858.64it/s]


In [5]:
brisk = cv2.BRISK_create(30)

In [6]:
for image_path in tqdm(image_paths):
    im = cv2.imread(image_path)
    kpts, des = brisk.detectAndCompute(im, None)
    des_list.append((image_path, des))   

100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:48<00:00,  4.35it/s]


In [7]:
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][1]
for image_path, descriptor in tqdm(des_list[1:]):
    descriptors = np.vstack((descriptors, descriptor))  


100%|████████████████████████████████████████████████████████████████████████████████| 209/209 [00:07<00:00, 27.60it/s]


In [8]:
#kmeans works only on float, so convert integers to float
descriptors_float = descriptors.astype(float)  

In [9]:
# Perform k-means clustering and vector quantization
from scipy.cluster.vq import kmeans, vq

k = 200  #k means with 100 clusters 
voc, variance = kmeans(descriptors_float, k, 1) 

In [12]:
# Calculate the histogram of features and represent them as vector
#vq Assigns codes from a code book to observations.
im_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
    words, distance = vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w] += 1

In [13]:

# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')

In [14]:

# Scaling the words
#Standardize features by removing the mean and scaling to unit variance
#In a way normalization
from sklearn.preprocessing import StandardScaler
stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)

In [15]:
#Train an algorithm to discriminate vectors corresponding to positive and negative training images
# Train the Linear SVM
from sklearn.svm import LinearSVC
clf = LinearSVC(max_iter=10000)  #Default of 100 is not converging
clf.fit(im_features, np.array(image_classes))

LinearSVC(max_iter=10000)

In [16]:
# Save the SVM
#Joblib dumps Python object into one file
# from sklearn.externals import joblib
import joblib
joblib.dump((clf, training_names, stdSlr, k, voc), "bovw.pkl", compress=3)  

['bovw.pkl']